# Pembuat Data Sintetis untuk Catatan Servis Aset

Notebook ini bertujuan untuk membuat data sintetis berdasarkan sampel data catatan servis aset yang diberikan. Proses ini melibatkan analisis mendalam terhadap data asli untuk memahami pola, distribusi, dan hubungan antar variabel. Data sintetis yang dihasilkan diharapkan memiliki kualitas yang baik dan didasarkan pada alasan ilmiah (data-driven) sehingga dapat berguna untuk berbagai keperluan seperti pengujian, pengembangan, atau simulasi.

**Langkah-langkah utama:**
1.  **Muat dan Analisis Data Asli:** Memahami karakteristik data sumber.
2.  **Definisi Fungsi Pembuatan Data Sintetis:** Merancang logika untuk menghasilkan data baru.
3.  **Pembuatan dan Tinjauan Data Sintetis:** Menghasilkan dan memeriksa data yang telah dibuat.

### Impor Library yang Dibutuhkan

Sel ini mengimpor semua pustaka Python yang akan kita gunakan dalam notebook ini.
- `pandas` untuk manipulasi data tabular.
- `numpy` untuk operasi numerik, terutama untuk menghasilkan angka acak dari distribusi tertentu.
- `datetime`, `timedelta` dari modul `datetime` untuk bekerja dengan tanggal dan waktu.
- `random` untuk beberapa pilihan acak lainnya.
- `StringIO` dari modul `io` untuk membaca string CSV seolah-olah itu adalah file.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from io import StringIO

### Data Asli

Di sel ini, kita akan mendefinisikan data asli dalam format string CSV. Ini adalah data yang akan kita analisis untuk membuat model data sintetis.

In [2]:
df = pd.read_csv('AD1489HR.csv')
df.head()

,Action Date,Asset,Parts & Services,Action,Current Odometer
0,2024-06-27 07:00:00,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / FILTER SOLAR,Service,"973,253.00"
1,2024-05-30 07:00:00,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / FILTER SOLAR,Service,"973,252.00"
2,2024-04-01 11:41:14,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR ATAS,Deactivate,"964,170.00"
3,2024-04-01 11:41:14,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR BAWAH,Deactivate,"964,170.00"
4,2024-03-21 22:49:52,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / FILTER SOLAR,Reset,"999,106.00"


### Langkah 1: Fungsi untuk Memuat dan Menganalisis Data Asli

Fungsi `muat_dan_analisis_data` di bawah ini akan melakukan beberapa tugas:
1.  **Memuat Data**: Membaca data CSV dari string ke dalam DataFrame pandas.
2.  **Pembersihan dan Konversi Tipe Data**:
    *   Mengubah kolom `Action Date` menjadi tipe `datetime`.
    *   Mengubah kolom `Current Odometer` menjadi tipe numerik (float), setelah menghilangkan karakter koma.
3.  **Pengurutan Data**: Mengurutkan data berdasarkan `Action Date` secara menaik (ascending) untuk memudahkan analisis progresi.
4.  **Analisis Statistik**:
    *   Menghitung rata-rata dan standar deviasi **waktu antar kejadian** (dalam detik).
    *   Menghitung rata-rata dan standar deviasi **kenaikan odometer** antar kejadian "normal" (yaitu, kejadian yang bukan merupakan reset odometer ke nilai tinggi seperti `99X,XXX`). Ini penting untuk mensimulasikan penggunaan aset yang realistis.
    *   Menganalisis **frekuensi relatif** dari berbagai jenis `Action` (misalnya, 'Service', 'Reset', 'Deactivate') untuk setiap kategori `Parts & Services` utama.
    *   Menganalisis **probabilitas kemunculan** jenis kejadian utama (misalnya, "PM / FILTER SOLAR" vs. "PM / OTHERS / FILTER SOLAR ATAS/BAWAH").
    *   Menganalisis **nilai odometer tipikal** yang terkait dengan aksi "Reset" yang menghasilkan odometer tinggi (misalnya, `99X,XXX`).
5.  **Menentukan Kondisi Awal**: Mengidentifikasi tanggal mulai dan odometer awal dari data asli untuk digunakan sebagai titik awal pembuatan data sintetis.

Hasil analisis ini akan disimpan dalam sebuah dictionary `analysis` yang akan digunakan oleh fungsi pembuat data sintetis.

In [6]:
csv_file_content = """Action Date,Asset,Parts & Services,Action,Current Odometer
2024-06-27 07:00:00,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / FILTER SOLAR,Service,"973,253.00"
2024-05-30 07:00:00,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / FILTER SOLAR,Service,"973,252.00"
2024-04-01 11:41:14,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR ATAS,Deactivate,"964,170.00"
2024-04-01 11:41:14,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR BAWAH,Deactivate,"964,170.00"
2024-03-21 22:49:52,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / FILTER SOLAR,Reset,"999,106.00"
2024-03-21 22:10:11,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / FILTER SOLAR,Reset,"999,106.00"
2024-03-19 23:53:24,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR ATAS,Deactivate,"998,778.00"
2024-03-19 23:53:24,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR BAWAH,Deactivate,"998,778.00"
2024-03-19 23:52:41,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / FILTER SOLAR,Reset,"998,778.00"
2024-03-19 22:47:13,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / FILTER SOLAR,Reset,"998,778.00"
2024-03-12 16:26:24,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR ATAS,Reset,"999,651.00"
2024-03-12 16:26:24,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR BAWAH,Reset,"999,651.00"
2024-03-12 08:37:41,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR ATAS,Reset,"999,651.00"
2024-03-12 08:37:41,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR BAWAH,Reset,"999,651.00"
2024-03-10 22:59:16,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR ATAS,Reset,"959,337.00"
2024-03-10 22:59:15,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR BAWAH,Reset,"959,337.00"
2024-03-09 22:41:16,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR ATAS,Reset,"959,076.00"
2024-03-09 22:41:16,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR BAWAH,Reset,"959,076.00"
2024-03-08 22:41:15,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR ATAS,Reset,"958,751.00"
2024-03-08 22:41:15,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR BAWAH,Reset,"958,751.00"
2023-11-02 07:00:00,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / FILTER SOLAR,Service,"923,236.00"
"""

print("Data asli telah didefinisikan.")

Data asli telah didefinisikan.


In [18]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from io import StringIO # Pastikan StringIO diimpor jika belum

# Jika Anda menjalankan ini di notebook dan pandas serta numpy sudah diimpor di sel sebelumnya,
# Anda tidak perlu mengimpornya lagi di sini.

def muat_dan_analisis_data(csv_content):
    # Membaca string CSV ke DataFrame
    df = pd.read_csv(StringIO(csv_content))
    # Pembersihan dan Konversi Tipe Data
    df['Action Date'] = pd.to_datetime(df['Action Date'])
    df['Current Odometer'] = df['Current Odometer'].str.replace(',', '').astype(float)

    # Mengurutkan data berdasarkan tanggal untuk menganalisis progresi
    df = df.sort_values(by='Action Date').reset_index(drop=True)

    # Dictionary untuk menyimpan hasil analisis
    analysis = {}

    # Nama Aset (diasumsikan tunggal dari sampel)
    analysis['asset_name'] = df['Asset'].iloc[0] if not df.empty else "ASET_SINTETIS_DEFAULT"

    # Analisis Waktu Antar Kejadian
    df['Time Diff'] = df['Action Date'].diff().dt.total_seconds()
    analysis['avg_time_diff_seconds'] = df['Time Diff'].iloc[1:].mean()
    analysis['std_time_diff_seconds'] = df['Time Diff'].iloc[1:].std()
    if pd.isna(analysis['std_time_diff_seconds']):
        analysis['std_time_diff_seconds'] = analysis['avg_time_diff_seconds'] * 0.1 if not pd.isna(analysis['avg_time_diff_seconds']) else 3600*24
    if pd.isna(analysis['avg_time_diff_seconds']):
        analysis['avg_time_diff_seconds'] = 3600*24*7

    # Analisis Kenaikan Odometer (untuk operasi "normal")
    normal_odo_df = df[df['Current Odometer'] < 990000].copy()
    if not normal_odo_df.empty and len(normal_odo_df) > 1:
        normal_odo_df['Odo Diff'] = normal_odo_df['Current Odometer'].diff()
        valid_odo_diffs = normal_odo_df['Odo Diff'].iloc[1:][normal_odo_df['Odo Diff'].iloc[1:] > 0]
        analysis['avg_odo_increment'] = valid_odo_diffs.mean()
        analysis['std_odo_increment'] = valid_odo_diffs.std()
        
        if pd.isna(analysis['avg_odo_increment']) or analysis['avg_odo_increment'] <= 0:
            analysis['avg_odo_increment'] = 50.0
        if pd.isna(analysis['std_odo_increment']):
            analysis['std_odo_increment'] = analysis['avg_odo_increment'] * 0.2
    else:
        analysis['avg_odo_increment'] = 50.0 
        analysis['std_odo_increment'] = 10.0

    # Analisis kombinasi Parts & Services dan Action serta frekuensinya
    ps_filter_solar_df = df[df['Parts & Services'] == 'PM / FILTER SOLAR']
    if not ps_filter_solar_df.empty:
        ps_filter_solar_actions = ps_filter_solar_df['Action'].value_counts(normalize=True).to_dict()
    else:
        ps_filter_solar_actions = {'Service': 0.5, 'Reset': 0.5} # Fallback
    analysis['ps_filter_solar_actions'] = ps_filter_solar_actions

    ps_others_atas_df = df[df['Parts & Services'] == 'PM / OTHERS / FILTER SOLAR ATAS']
    if not ps_others_atas_df.empty:
        ps_others_actions = ps_others_atas_df['Action'].value_counts(normalize=True).to_dict()
    else:
        ps_others_actions = {'Deactivate': 0.5, 'Reset': 0.5} # Fallback
    analysis['ps_others_actions'] = ps_others_actions

    count_filter_solar = df[df['Parts & Services'] == 'PM / FILTER SOLAR']['Action Date'].nunique()
    count_others = df[df['Parts & Services'] == 'PM / OTHERS / FILTER SOLAR ATAS']['Action Date'].nunique()

    total_unique_event_types = count_filter_solar + count_others
    if total_unique_event_types > 0:
        analysis['prob_filter_solar_event'] = count_filter_solar / total_unique_event_types
    else:
        analysis['prob_filter_solar_event'] = 0.5 # Fallback

    # Analisis nilai Odometer untuk aksi "Reset" yang menghasilkan odometer tinggi
    # Ini akan digunakan sebagai mean dan std untuk odometer tinggi secara umum.
    reset_odo_values = df[(df['Action'] == 'Reset') & (df['Current Odometer'] > 990000)]['Current Odometer']
    analysis['reset_odo_mean'] = reset_odo_values.mean() if not reset_odo_values.empty else 999000.0
    analysis['reset_odo_std'] = reset_odo_values.std() if not reset_odo_values.empty else 500.0
    if pd.isna(analysis['reset_odo_std']): analysis['reset_odo_std'] = 500.0
    if pd.isna(analysis['reset_odo_mean']): analysis['reset_odo_mean'] = 999000.0

    # --- AWAL PERUBAHAN ---
    # Analisis probabilitas odometer tinggi untuk kombinasi spesifik

    # 1. Untuk PM / OTHERS dengan Action == 'Reset'
    df_others_reset = df[(df['Parts & Services'].str.contains('PM / OTHERS / FILTER SOLAR', na=False)) & (df['Action'] == 'Reset')]
    if not df_others_reset.empty:
        high_odo_on_others_reset_count = (df_others_reset['Current Odometer'] > 990000).sum()
        total_others_reset_count = len(df_others_reset)
        analysis['prob_high_odo_on_others_reset'] = high_odo_on_others_reset_count / total_others_reset_count if total_others_reset_count > 0 else 0.5 # Fallback
    else:
        analysis['prob_high_odo_on_others_reset'] = 0.5 # Fallback jika tidak ada 'Reset' pada 'OTHERS'

    # 2. Untuk PM / OTHERS dengan Action == 'Deactivate'
    df_others_deactivate = df[(df['Parts & Services'].str.contains('PM / OTHERS / FILTER SOLAR', na=False)) & (df['Action'] == 'Deactivate')]
    if not df_others_deactivate.empty:
        high_odo_on_others_deactivate_count = (df_others_deactivate['Current Odometer'] > 990000).sum()
        total_others_deactivate_count = len(df_others_deactivate)
        analysis['prob_high_odo_on_others_deactivate'] = high_odo_on_others_deactivate_count / total_others_deactivate_count if total_others_deactivate_count > 0 else 0.2 # Fallback
    else:
        analysis['prob_high_odo_on_others_deactivate'] = 0.2 # Fallback jika tidak ada 'Deactivate' pada 'OTHERS'

    # 3. Untuk PM / FILTER SOLAR dengan Action == 'Reset'
    df_filter_solar_reset = df[(df['Parts & Services'] == 'PM / FILTER SOLAR') & (df['Action'] == 'Reset')]
    if not df_filter_solar_reset.empty:
        high_odo_on_filter_solar_reset_count = (df_filter_solar_reset['Current Odometer'] > 990000).sum()
        total_filter_solar_reset_count = len(df_filter_solar_reset)
        analysis['prob_high_odo_on_filter_solar_reset'] = high_odo_on_filter_solar_reset_count / total_filter_solar_reset_count if total_filter_solar_reset_count > 0 else 0.9 # Fallback
    else:
        analysis['prob_high_odo_on_filter_solar_reset'] = 0.9 # Fallback, asumsikan sering tinggi
    # --- AKHIR PERUBAHAN ---

    # Titik awal untuk data sintetis
    first_normal_odo_series = df[df['Current Odometer'] < 990000]['Current Odometer']
    if not first_normal_odo_series.empty:
        analysis['initial_odo'] = first_normal_odo_series.iloc[0]
    elif not df.empty:
        analysis['initial_odo'] = df['Current Odometer'].min()
    else:
        analysis['initial_odo'] = 1000.0 # Fallback odometer awal

    analysis['start_date'] = df['Action Date'].min() if not df.empty else datetime.now() - timedelta(days=365)

    return analysis

print("Fungsi muat_dan_analisis_data telah didefinisikan dengan perbaikan.")

Fungsi muat_dan_analisis_data telah didefinisikan dengan perbaikan.


### Menjalankan Analisis Data Asli

Sekarang kita akan memanggil fungsi `muat_dan_analisis_data` dengan konten CSV yang telah kita definisikan. Hasil analisis akan disimpan dalam variabel `analysis_results` dan kemudian ditampilkan. Ini penting untuk memverifikasi bahwa parameter untuk pembuatan data sintetis telah diekstrak dengan benar.

In [19]:
print("--- Menganalisis Data Asli ---")
analysis_results = muat_dan_analisis_data(csv_file_content)

print("\nParameter Hasil Analisis:")
for key, value in analysis_results.items():
    print(f"- {key}: {value}")

--- Menganalisis Data Asli ---

Parameter Hasil Analisis:
- asset_name: AD1489HR - HINO FL8JNKA-GGJ (FL235JN)
- avg_time_diff_seconds: 1028160.0
- std_time_diff_seconds: 2650222.0686741867
- avg_odo_increment: 8336.166666666666
- std_odo_increment: 13786.801926722046
- ps_filter_solar_actions: {'Reset': 0.5714285714285714, 'Service': 0.42857142857142855}
- ps_others_actions: {'Reset': 0.7142857142857143, 'Deactivate': 0.2857142857142857}
- prob_filter_solar_event: 0.5
- reset_odo_mean: 999296.5
- reset_odo_std: 398.7383674971418
- prob_high_odo_on_others_reset: 0.4
- prob_high_odo_on_others_deactivate: 0.5
- prob_high_odo_on_filter_solar_reset: 1.0
- initial_odo: 923236.0
- start_date: 2023-11-02 07:00:00


___

### Langkah 2: Fungsi untuk Membuat Data Sintetis

Fungsi `buat_data_sintetis` akan menghasilkan data baru baris per baris berdasarkan parameter yang telah diekstrak dari analisis data asli.

**Logika Utama:**
1.  **Inisialisasi**: Memulai dengan `current_date` dan `logical_current_odometer` dari hasil analisis. `logical_current_odometer` adalah odometer yang terus bertambah secara normal dan digunakan untuk menghitung odometer pada kejadian non-reset.
2.  **Loop Pembuatan**: Berulang hingga jumlah baris target tercapai.
3.  **Pembuatan `Action Date`**: Menambahkan delta waktu acak (berdasarkan `avg_time_diff_seconds` dan `std_time_diff_seconds` dari analisis) ke tanggal kejadian sebelumnya.
4.  **Pemilihan Jenis Kejadian**: Secara acak memilih apakah kejadian berikutnya adalah "PM / FILTER SOLAR" atau "PM / OTHERS / FILTER SOLAR ATAS/BAWAH" berdasarkan `prob_filter_solar_event`.
5.  **Pemilihan `Action`**: Memilih `Action` (misalnya, 'Service', 'Reset') berdasarkan probabilitas kondisional yang diamati untuk jenis `Parts & Services` yang terpilih.
6.  **Pembuatan `Current Odometer`**:
    *   Jika `Action` adalah 'Reset' dan jenis kejadiannya sesuai dengan pola reset odometer tinggi (misalnya, `PM / FILTER SOLAR` yang di-reset ke `99X,XXX`), maka odometer akan diambil dari distribusi normal berdasarkan `reset_odo_mean` dan `reset_odo_std`.
    *   Jika tidak, odometer akan dihitung dengan menambahkan kenaikan acak (berdasarkan `avg_odo_increment` dan `std_odo_increment`) ke `logical_current_odometer`. `logical_current_odometer` kemudian diperbarui.
7.  **Penanganan Kejadian Berpasangan**: Jika kejadian "PM / OTHERS" terpilih, dua baris akan dibuat (satu untuk ATAS, satu untuk BAWAH) dengan `Action Date`, `Action`, dan `Current Odometer` yang sama.
8.  **Format Output**: Memastikan `Current Odometer` diformat sebagai string dengan koma seperti pada data asli.

Fungsi ini mengembalikan DataFrame pandas yang berisi data sintetis.

In [20]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
# Pastikan pandas, numpy, datetime, timedelta, dan random sudah diimpor jika ini sel terpisah.

def buat_data_sintetis(analysis, num_rows_target):
    synthetic_records = []
    
    current_date = analysis['start_date']
    logical_current_odometer = analysis['initial_odo'] 

    def choose_action(action_probs):
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        if len(probs) > 0 and not np.isclose(sum(probs), 1.0) and sum(probs) > 0: # Tambahkan sum(probs) > 0
            probs = [p / sum(probs) for p in probs]
        elif len(probs) > 0 and sum(probs) == 0 and len(actions) > 0: # Jika semua probabilitas 0, pilih secara acak
            return random.choice(actions)


        if not actions:
            return random.choice(['Service', 'Reset', 'Deactivate'])
        if not probs or len(actions) != len(probs):
            return random.choice(actions) if actions else random.choice(['Service', 'Reset', 'Deactivate'])
        
        # Jika setelah normalisasi masih ada masalah (misal, semua probabilitas jadi 0 karena pembulatan)
        if np.isclose(sum(probs), 0.0) and len(actions) > 0:
            return random.choice(actions)

        return np.random.choice(actions, p=probs)

    rows_generated = 0
    while rows_generated < num_rows_target:
        time_delta_seconds = abs(np.random.normal(
            analysis['avg_time_diff_seconds'], 
            analysis['std_time_diff_seconds']
        ))
        time_delta_seconds = max(3600, time_delta_seconds)
        current_date += timedelta(seconds=time_delta_seconds)
        
        is_filter_solar_event_type = random.random() < analysis['prob_filter_solar_event']
        
        # current_event_odometer akan ditentukan di bawah
        action_for_event = ""

        odo_increment_for_event = abs(np.random.normal(analysis['avg_odo_increment'], analysis['std_odo_increment']))
        odo_increment_for_event = max(1.0, odo_increment_for_event)

        if is_filter_solar_event_type:
            parts_services = "PM / FILTER SOLAR"
            action_for_event = choose_action(analysis['ps_filter_solar_actions'])
            
            # --- AWAL PERUBAHAN UNTUK PM / FILTER SOLAR ---
            use_high_odo = False
            if action_for_event == 'Reset':
                # Gunakan probabilitas dari analisis, dengan fallback jika kunci tidak ada
                prob_high = analysis.get('prob_high_odo_on_filter_solar_reset', 0.9) # Default fallback 90%
                if random.random() < prob_high:
                    use_high_odo = True
            
            if use_high_odo:
                 current_event_odometer = round(abs(np.random.normal(analysis['reset_odo_mean'], analysis['reset_odo_std'])))
            else: 
                logical_current_odometer += odo_increment_for_event
                current_event_odometer = round(logical_current_odometer)
            # --- AKHIR PERUBAHAN UNTUK PM / FILTER SOLAR ---

            synthetic_records.append({
                "Action Date": current_date.strftime('%Y-%m-%d %H:%M:%S'),
                "Asset": analysis['asset_name'],
                "Parts & Services": parts_services,
                "Action": action_for_event,
                "Current Odometer": f"{int(current_event_odometer):,}.00"
            })
            rows_generated += 1

        else: # Kejadian PM / OTHERS / FILTER SOLAR ATAS/BAWAH
            action_for_event = choose_action(analysis['ps_others_actions'])
            
            # --- AWAL PERUBAHAN UNTUK PM / OTHERS ---
            use_high_odo_for_others = False
            if action_for_event == 'Reset':
                # Gunakan probabilitas dari analisis, dengan fallback jika kunci tidak ada
                prob_high = analysis.get('prob_high_odo_on_others_reset', 0.5) # Default fallback 50%
                if random.random() < prob_high:
                    use_high_odo_for_others = True
            elif action_for_event == 'Deactivate':
                # Gunakan probabilitas dari analisis, dengan fallback jika kunci tidak ada
                prob_high = analysis.get('prob_high_odo_on_others_deactivate', 0.2) # Default fallback 20%
                if random.random() < prob_high:
                    use_high_odo_for_others = True
            
            if use_high_odo_for_others:
                current_event_odometer = round(abs(np.random.normal(analysis['reset_odo_mean'], analysis['reset_odo_std'])))
            else: 
                logical_current_odometer += odo_increment_for_event
                current_event_odometer = round(logical_current_odometer)
            # --- AKHIR PERUBAHAN UNTUK PM / OTHERS ---

            parts_atas = "PM / OTHERS / FILTER SOLAR ATAS"
            parts_bawah = "PM / OTHERS / FILTER SOLAR BAWAH"
            
            synthetic_records.append({
                "Action Date": current_date.strftime('%Y-%m-%d %H:%M:%S'),
                "Asset": analysis['asset_name'],
                "Parts & Services": parts_atas,
                "Action": action_for_event,
                "Current Odometer": f"{int(current_event_odometer):,}.00"
            })
            rows_generated += 1
            if rows_generated >= num_rows_target: break

            synthetic_records.append({
                "Action Date": current_date.strftime('%Y-%m-%d %H:%M:%S'), 
                "Asset": analysis['asset_name'],
                "Parts & Services": parts_bawah,
                "Action": action_for_event, 
                "Current Odometer": f"{int(current_event_odometer):,}.00" 
            })
            rows_generated += 1
        
    return pd.DataFrame(synthetic_records)

print("Fungsi buat_data_sintetis telah didefinisikan dengan perbaikan.")

Fungsi buat_data_sintetis telah didefinisikan dengan perbaikan.


___
### Langkah 3: Membuat dan Menampilkan Data Sintetis

Di sel ini, kita akan:
1.  Menentukan jumlah baris data sintetis yang ingin kita hasilkan.
2.  Memanggil fungsi `buat_data_sintetis` dengan parameter hasil analisis dan jumlah baris yang diinginkan.
3.  Menampilkan beberapa baris pertama dari data sintetis yang dihasilkan untuk verifikasi.
4.  (Opsional) Menyimpan data sintetis ke dalam file CSV.

In [23]:
# Tentukan jumlah baris data sintetis yang diinginkan
# Ingat bahwa kejadian "OTHERS" menghasilkan 2 baris, jadi target ini adalah perkiraan.
num_synthetic_rows_target = 50

print(f"--- Membuat {num_synthetic_rows_target} Baris Data Sintetis (Target) ---")
synthetic_df = buat_data_sintetis(analysis_results, num_synthetic_rows_target)

# print(f"\nTotal baris data sintetis yang berhasil dibuat: {len(synthetic_df)}")
# print("\nContoh Data Sintetis (15 baris pertama):")
# print(synthetic_df.head(15))
data_baru = pd.DataFrame(synthetic_df)
data_baru
# (Opsional) Simpan data sintetis ke file CSV
# synthetic_df.to_csv("data_servis_sintetis.csv", index=False)
# print("\nData sintetis telah disimpan ke file data_servis_sintetis.csv")

--- Membuat 50 Baris Data Sintetis (Target) ---


,Action Date,Asset,Parts & Services,Action,Current Odometer
0,2023-11-07 08:59:50,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR ATAS,Deactivate,"933,520.00"
1,2023-11-07 08:59:50,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR BAWAH,Deactivate,"933,520.00"
2,2023-12-03 15:18:33,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / FILTER SOLAR,Service,"936,654.00"
3,2024-01-08 03:54:11,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / FILTER SOLAR,Reset,"998,792.00"
4,2024-01-25 11:34:21,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR ATAS,Deactivate,"998,833.00"
5,2024-01-25 11:34:21,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR BAWAH,Deactivate,"998,833.00"
6,2024-02-23 01:22:33,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR ATAS,Reset,"959,724.00"
7,2024-02-23 01:22:33,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR BAWAH,Reset,"959,724.00"
8,2024-03-14 20:06:55,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / FILTER SOLAR,Service,"962,895.00"
9,2024-03-26 18:48:08,AD1489HR - HINO FL8JNKA-GGJ (FL235JN),PM / OTHERS / FILTER SOLAR ATAS,Reset,"977,030.00"
